# **02 - DataCleaning**

## Objectives

* Check for missing values in the dataset
* Address missing values in the dataset
* Implement the data cleaning process

## Inputs

* outputs/datasets/collection/HousePricesRecords.csv

## Outputs

* outputs/datasets/cleaned/train_set_cleaned.csv
* outputs/datasets/cleaned/test_set_cleaned.csv
* outputs/datasets/cleaned/HousePricesCleaned.csv

## Additional Comments

* The decision to impute or drop variables with missing data should be made carefully because both options can significantly impact the quality and integrity of your model.


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

Uses pandas library to load dataset into DataFrames df and displays the first 10 rows of the dataset.

In [ ]:
import pandas as pd
df = pd.read_csv("outputs/datasets/collection/HousePricesRecords.csv")
df.head(10)

Identifies and extracts the names of columns in the DataFrame df that contain missing (NaN) values.

In [ ]:
columns_with_missing_values = df.columns[df.isna().sum() > 0].to_list()
columns_with_missing_values

Returns the number of elements in the columns_with_missing_values list.

In [ ]:
len(columns_with_missing_values)

Summary only for the columns in columns_with_missing_values.

In [ ]:
df[columns_with_missing_values].info()

If columns_with_missing_values exist, generate a minimal data profile report using the ydata_profiling library. If no columns with missing data are found, print a message indicating there are no columns with missing values.

In [ ]:
from ydata_profiling import ProfileReport

if columns_with_missing_values:
    profile = ProfileReport(df=df[columns_with_missing_values], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no columns with missing data")

Analyzes missing data in a DataFrame by calculating the number of missing values and the percentage of missing data for each column. Creates a new DataFrame containing the count of missing values, the percentage of missing data, and the data type for each column. This new DataFrame is sorted by the percentage of missing data in descending order and filtered to include only columns with missing data.

In [9]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=['PercentageOfDataset'], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data

In [ ]:
EvaluateMissingData(df)

Visualizes the impact of data cleaning by comparing the distribution of specified variables in original and cleaned datasets. It generates bar plots for categorical variables and histograms for numerical ones, to assess the effect of data cleaning visually.

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline


sns.set(style="whitegrid")

def DataCleaningEffect(df_original, df_cleaned, variables_applied_with_method):
    """
    Function to visualize data cleaning effect
    """
    flag_count = 1

    # Identify categorical variables
    categorical_variables = df_original.select_dtypes(exclude=['number']).columns

    # Loop through each variable in the list provided
    for var in variables_applied_with_method:
        print("\n=====================================================================================")
        print(f"* Distribution Effect Analysis After Data Cleaning Method on variable: {var}")

        if var in categorical_variables:
            # For categorical variables, create a bar plot
            df1 = pd.DataFrame({"Type": "Original", "Value": df_original[var]})
            df2 = pd.DataFrame({"Type": "Cleaned", "Value": df_cleaned[var]})
            dfAux = pd.concat([df1, df2], axis=0)
            dfAux.reset_index(drop=True, inplace=True)  # Reset the index to avoid duplicates

            fig, axes = plt.subplots(figsize=(15, 5))
            sns.countplot(data=dfAux, x="Value", hue="Type", palette=['#1f77b4', "#ff7f0e"])
            axes.set_title(f"Distribution Plot {flag_count}: {var}")
            plt.xticks(rotation=90)
            plt.legend()
            plt.show()
            print(f"Displaying bar plot for categorical variable: {var}")

        else:
            # For numerical variables, create histograms
            fig, axes = plt.subplots(figsize=(10, 5))
            sns.histplot(data=df_original, x=var, color="#1f77b4", label='Original', kde=True, element="step", ax=axes)
            sns.histplot(data=df_cleaned, x=var, color="#ff7f0e", label='Cleaned', kde=True, element="step", ax=axes)
            axes.set_title(f"Distribution Plot {flag_count}: {var}")
            plt.legend()
            plt.show()
            print(f"Displaying histogram for numerical variable: {var}")

        plt.close(fig)
        flag_count += 1

Splits a DataFrame into training and testing sets for machine learning purposes, where TrainSet contains 80% of the data and TestSet contains 20%.

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=42)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

Evaluates and prints information about missing data in the TrainSet DataFrame. The number of variables with missing data is printed, followed by the details of these variables. Additionally, it prints the names of all columns present in the TrainSet DataFrame.

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
print(df_missing_data)

print(f"TrainSet columns: {TrainSet.columns}")

Remove columns EnclosedPorch and WoodDeckSF, from the TrainSet DataFrame. They both have an extreme amount of missing data.

In [ ]:
from feature_engine.selection import DropFeatures

variables_to_drop=['EnclosedPorch' , 'WoodDeckSF']
imputer = DropFeatures(features_to_drop=variables_to_drop)
df_method = imputer.fit_transform(TrainSet)

for i in variables_to_drop:
    print(i in df_method.columns.to_list())

Applies mean imputation to the numerical variables 'LotFrontage' and 'BedroomAbvGr' in the dataset TrainSet using MeanMedianImputer. The cleaned dataset df_method is passed to the DataCleaningEffect function, which visualizes the impact of imputation by generating histograms for numerical variables.

In [ ]:
from feature_engine.imputation import MeanMedianImputer

variables_mean = ['LotFrontage' , 'BedroomAbvGr']
imputer = MeanMedianImputer(imputation_method='mean', variables=variables_mean)
df_method = imputer.fit_transform(TrainSet)
DataCleaningEffect(df_original=TrainSet,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_mean)

Applies median imputation to the numerical variables '2ndFlrSF' and 'MasVnrArea' in the dataset TrainSet using MeanMedianImputer. The cleaned dataset df_method is passed to the DataCleaningEffect function, which visualizes the impact of imputation by generating histograms for numerical variables.

In [ ]:
variables_median = ['2ndFlrSF' , 'MasVnrArea']
imputer = MeanMedianImputer(imputation_method='median', variables=variables_median)
df_method = imputer.fit_transform(TrainSet)
DataCleaningEffect(df_original=TrainSet,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_median)

Applies categorical imputation to the categorical variables 'GarageFinish' and 'BsmtFinType1' in the dataset TrainSet using CategoricalImputer. The cleaned dataset df_method is passed to the DataCleaningEffect function, which visualizes the impact of imputation by generating bar plots for categorical variables.

In [ ]:
from feature_engine.imputation import CategoricalImputer

variables_categorical = ['GarageFinish' , 'BsmtFinType1']
imputer = CategoricalImputer(imputation_method='missing', fill_value='None', variables=variables_categorical)
df_method = imputer.fit_transform(TrainSet)
DataCleaningEffect(df_original=TrainSet,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_categorical)

The cleaning is implemented on Test, Train, and df, followed by a verification to ensure that none contain missing data.

In [18]:
from sklearn.impute import SimpleImputer

median_imputer = SimpleImputer(strategy='median')

numerical_cols_median = ['2ndFlrSF', 'MasVnrArea']

TrainSet[numerical_cols_median] = median_imputer.fit_transform(TrainSet[numerical_cols_median])
TestSet[numerical_cols_median] = median_imputer.transform(TestSet[numerical_cols_median])
df[numerical_cols_median] = median_imputer.transform(df[numerical_cols_median])

variables_to_drop = ['EnclosedPorch', 'WoodDeckSF', 'GarageFinish', 'BsmtFinType1', 'BsmtExposure', 'GarageYrBlt']

TrainSet = TrainSet.drop(columns=[col for col in variables_to_drop if col in TrainSet.columns], errors='ignore')
TestSet = TestSet.drop(columns=[col for col in variables_to_drop if col in TestSet.columns], errors='ignore')
df = df.drop(columns=[col for col in variables_to_drop if col in df.columns], errors='ignore')

mean_imputer = SimpleImputer(strategy='mean')

mean_imputation_cols = ['LotFrontage', 'BedroomAbvGr']

TrainSet[mean_imputation_cols] = mean_imputer.fit_transform(TrainSet[mean_imputation_cols])
TestSet[mean_imputation_cols] = mean_imputer.transform(TestSet[mean_imputation_cols])
df[mean_imputation_cols] = mean_imputer.transform(df[mean_imputation_cols])

Prints the count of missing (null) values in three datasets (TrainSet, TestSet, and df) after cleaning.

In [ ]:
print("Missing values in TrainSet after cleaning:")
print(TrainSet.isnull().sum())

print("Missing values in TestSet after cleaning:")
print(TestSet.isnull().sum())

print("Missing values in df after cleaning:")
print(df.isnull().sum())

TrainSet is passed to the EvaluateMissingData function.

In [ ]:
EvaluateMissingData(TrainSet)

TestSet is passed to the EvaluateMissingData function.

In [ ]:
EvaluateMissingData(TestSet)

df is passed to the EvaluateMissingData function.

In [ ]:
EvaluateMissingData(df)

Create the directory structure outputs/datasets/cleaned

In [ ]:
import os
try:
  os.makedirs("outputs/datasets/cleaned")
except Exception as e:
  print(e)

Save TrainSet as a CSV file.

In [24]:
TrainSet.to_csv("outputs/datasets/cleaned/train_set_cleaned.csv", index=False)

Save TestSet as a CSV file.

In [25]:
TestSet.to_csv("outputs/datasets/cleaned/test_set_cleaned.csv", index=False)

Save df as a CSV file.

In [26]:
df.to_csv("outputs/datasets/cleaned/HousePricesCleaned.csv", index=False)

Checks for missing values in the df DataFrame.

In [ ]:
import pandas as pd

df = pd.read_csv("outputs/datasets/cleaned/HousePricesCleaned.csv")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])